In [3]:
# TensorFlow について基本的な考え方を学んでいく

# いつもの設定
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "tensorflow"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [4]:
# tensorFlow は計算グラフの構築フェーズとの実行フェーズに分かれている。構築フェーズをpythonで実行しても変数初期化等は一切お行われない。

# 構築フェーズ：機械学習モデルを訓練するために必要な計算グラフを構築する
# 実行フェーズ：訓練のステップを繰り返してモデルパラメータを改良するループを作成する

import tensorflow as tf

# 作成したノードをデフォルトグラフをリセットする。開発/検証環境のお決まりコードかな？
reset_graph()
# 以下を実行しても初期化さえされていない
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

In [7]:
# ここでx,y,f を表示してみても属性が表示されるだけ


Tensor("add_1:0", shape=(), dtype=int32)


In [8]:
# 一番初歩的な実行フェーズの書き方
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


In [9]:
# 実行フェーズは以下のコードでクローズする必要がある
sess.close()

In [15]:
# 複数の初期化子を同時に初期化するコード
init = tf.global_variables_initializer()

In [16]:
# pythonシェルやjupyterでやる場合に簡単な方法⇒これで覚えていこう
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)

42


C:\Anaconda3\envs\python36\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [20]:
# この方法の注意点は以下のように手動でセッションを閉じる必要があること
sess.close()

In [21]:
# 作成したノードは自動的にデフォルトノードに追加される
# pythonシェルやjupyter上では同じコードを何度も実行するのでデフォルトコードに重複ノードが出来る。
# 毎回以下コードを実行するのがお勧め
tf.reset_default_graph()

In [26]:
# なお、グラフがあるかの判断は以下コードにて行う
reset_graph()

# tf.Variable() は変数宣言+初期化を行う　() は初期値を入れているだけ。名前を別に定義も出来たりする
# ここらの詳細は　https://deepage.net/tensorflow/2017/06/02/tensorflow-variable.html
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [23]:
# 複数の独立したグラフを管理したい場合の方法
graph = tf.Graph() # 新しいグラフを作る
with graph.as_default(): # with ブロックでそれを一時的なデフォルトグラフにする
    x2 = tf.Variable(2)

x2.graph is graph # ここで作られたグラフの存在確認

True

In [24]:
x2.graph is tf.get_default_graph() # 上記がデフォルトグラフに「なっていない」ことを確認

False

In [27]:
# ノードは　tensorflowではオペレーションと呼んでいる。計算や処理を行うブロックのこと。セッション単位で管理される？
# セッションは複数作ることが可能で、変数の値は、それぞれ別に保存される
# ノードの値のライフサイクル
# 下記コードは z 評価(実行時)に w と x を再度評価している。つまり y 評価時の結果を再利用していない
# ノードの値は

w = tf.constant(3) # tf.constant() は定数 # 参考は　https://deepage.net/tensorflow/2018/10/02/tensorflow-constant.html
# 変数ではないので注意！！
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess: # wtih session 内ではこれがデフォルトセッションになり、さらにclose() が要らなくなる
    print(y.eval())  # 10 eval()はtf.get_default_session().run() と同じ。つまりセッション開始命令
    print(z.eval())  # 15

10
15
